In [1]:
using PyCall

In [2]:
redis = pyimport("redis")
query = pyimport("redis.commands.search.query")
r = redis.Redis(host="localhost", port=6379)

PyObject Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [3]:
_docs = r.ft("b:nodes").search("*").docs
_docs

6-element Vector{PyObject}:
 PyObject Document {'id': 'b:n:70cd7da35abef4ec7d8670c16ce233be47a0c8d9', 'payload': None, 'sha1': '70cd7da35abef4ec7d8670c16ce233be47a0c8d9', 'searchable': '0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000010000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [5]:
_query = (
    query.Query("*").return_fields("sha1", "labels", "searchable").paging(0, 100)
)

_docs = r.ft("b:nodes").search(_query).docs
q_vector = _docs[1]["searchable"]

PyObject '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000001000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [6]:
stmt = raw"*=>[KNN 100 @searchable $vec as score]"
_query = (
    query.Query(stmt).sort_by("score").return_fields("sha1", "labels", "score").paging(0, 100).dialect(2)
)

query_params = Dict("vec" => q_vector)

r.ft("b:nodes").search(_query, query_params).docs

6-element Vector{PyObject}:
 PyObject Document {'id': 'b:n:70cd7da35abef4ec7d8670c16ce233be47a0c8d9', 'payload': None, 'score': '0', 'sha1': '70cd7da35abef4ec7d8670c16ce233be47a0c8d9', 'labels': 'node4'}
 PyObject Document {'id': 'b:n:5da690bfa7f1f0221f3bbcd581b33c9f7321e356', 'payload': None, 'score': '5.44171709104e-14', 'sha1': '5da690bfa7f1f0221f3bbcd581b33c9f7321e356', 'labels': 'node1'}
 PyObject Document {'id': 'b:n:0263a5ee90d2486ef7b786059101519dc7180ed9', 'payload': None, 'score': '5.44356633337e-14', 'sha1': '0263a5ee90d2486ef7b786059101519dc7180ed9', 'labels': 'node2'}
 PyObject Document {'id': 'b:n:bb558567ef9d0a4044ec8c633861dbc497afe315', 'payload': None, 'score': '1.08250126256e-13', 'sha1': 'bb558567ef9d0a4044ec8c633861dbc497afe315', 'labels': 'union'}
 PyObject Document {'id': 'b:n:0d6579afb0a1a2bb9d303378c9534e0a583ea8a3', 'payload': None, 'score': 'inf', 'sha1': '0d6579afb0a1a2bb9d303378c9534e0a583ea8a3', 'labels': 'node3'}
 PyObject Document {'id': 'b:n:b1321651af2

# References
1. https://juliapy.github.io/PythonCall.jl/stable/pythoncall/
2. https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/tags/ 